*This exercise shows ho to use the python API, rather than using the high-level PtychoRunnerScan classes, or the pynx-XXpty.py scripts*

In [ ]:
%matplotlib notebook
from pynx.ptycho import simulation, clptycho, shape
from pynx.utils.plot_utils import plot_llk_history
import numpy as np
from pylab import rcParams, figure
rcParams['figure.figsize'] = (12, 8)  # Figure size for inline display

**Simulation data**

In [ ]:
# Simulation of the ptychography data:
n=256 # Size of the frames
obj_info = {'type':'phase_ampl', 'phase_stretch':np.pi/2, 'alpha_win':.2}
probe_info = {'type':'focus', 'aperture': (60e-6, 200e-6), 'focal_length': .08, 'defocus': 100e-6,'shape': (n,n)}

#50 scan positions correspond to 4 turns, 78 to 5 turns, 113 to 6 turns
scan_info = {'type': 'spiral', 'scan_step_pix': 30, 'n_scans': 100}
data_info = {'num_phot_max': 1e9, 'bg': 0, 'wavelength': 1.5e-10, 'detector_dist': 1, 'detector_pixel_size': 55e-6, 'noise': 'poisson'}

# Initialisation of the simulation with specified parameters, specific <object>, <probe> or <scan>positions can be passed as:
# s = ptycho.Simulation(obj=<object>, probe=<probe>, scan = <scan>)
# omitting obj_info, probe_info or scan_info (or passing it as empty dictionary "{}")
s = simulation.Simulation(obj_info=obj_info, probe_info=probe_info, scan_info=scan_info, data_info=data_info)

# Data simulation: probe.show(), obj.show(), scan.show() and s.show_illumination_sum() will visualise the integrated total coverage of the beam
s.make_data()

pos = s.scan.values
ampl = s.amplitude.values # square root of the measured diffraction pattern intensity

**Prepare initial random object, and approximate probe to be optimized**

In [ ]:
nx,ny = shape.objShape(pos, ampl.shape[1:])

# Initial object
#obj_init_info = {'type':'flat','shape':(nx,ny)}
obj_init_info = {'type': 'random', 'range': (0,1,0,0.5), 'shape': (nx, ny)}
# Initial probe
probe_init_info = {'type':'focus', 'aperture': (50e-6, 180e-6), 'focal_length': .08, 'defocus': 50e-6,'shape': (n,n)}
data_info = {'wavelength': 1.5e-9, 'detector_dist': 1, 'detector_pixel_size': 55e-6}
init = simulation.Simulation(obj_info=obj_init_info, probe_info=probe_init_info, data_info=data_info)

init.make_obj()
init.make_probe()

**Ptycho object**

In [ ]:
pixel_size_object= 1e-8
p = clptycho.Ptycho2D(iobs= ampl**2, positions=pos, probe=s.probe.values.copy(), obj=init.obj.values.copy()
                      , opencl_device="K80", pixel_size_object=1e-8)

**Run algorithm. FIrst optimize only object, then probe, with different algorithms**

In [ ]:
p.run_difference_map(40, update_object=True, update_probe=False, verbose=20, doplot=True)

In [ ]:
p.run_difference_map(60, update_object=True, update_probe=True, verbose=20, doplot=True)

In [ ]:
p.run_alternating_projection(100, update_object=True, update_probe=True, verbose=20, doplot=True)

In [ ]:
p.run_ml_poisson(50, update_object=True, update_probe=True, verbose=20, doplot=True)

In [ ]:
%matplotlib inline
figure(figsize=(8,6))
plot_llk_history(p.history)